# 1. Initializations

## 1.1 General imports

In [ ]:
### data management
import pandas as pd
import numpy as np

### régression
import scipy.stats as stats
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression, RidgeCV, Lasso, LassoCV
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict
from sklearn.feature_selection import f_regression, SelectKBest, SelectFromModel
from sklearn.metrics import mean_squared_error

### graphical matplotlib basics
import matplotlib.pyplot as plt
# for jupyter notebook management
%matplotlib inline

### graphical seaborn basics
import seaborn as sns

## 1.2 General dataframe functions

In [ ]:
import smartcheck.dataframe_common as dfc

## 1.3 General classification functions

In [ ]:
# None

# 2. Loading and Data Quality

## 2.1 Loading of data sets and general exploration

In [ ]:
df_auto_raw = dfc.load_dataset_from_config('auto_data', sep=',')

if df_auto_raw is not None and isinstance(df_auto_raw, pd.DataFrame):
    # display(df_auto_raw.head())
    dfc.log_general_info(df_auto_raw)
    nb_first, nb_total = dfc.detect_and_log_duplicates_and_missing(df_auto_raw)
    if nb_first != nb_total:
        print(dfc.duplicates_index_map(df_auto_raw))
    df_auto = dfc.normalize_column_names(df_auto_raw)
    display(df_auto.head())

In [ ]:
df_auto_desc = df_auto.select_dtypes(include=np.number).describe()
display(df_auto_desc)
df_auto_cr = df_auto.select_dtypes(include=np.number).corr()
display(df_auto_cr)

## 2.2 Data quality refinement

In [ ]:
# Original backup and duplicates management
df_auto_orig = df_auto.copy()
df_auto = df_auto.drop_duplicates()
df_auto = df_auto[(df_auto.normalized_losses != '?') &
                  (df_auto.bore != '?') &
                  (df_auto.stroke != '?')]
df_auto.normalized_losses = df_auto.normalized_losses.astype(int)
df_auto.horsepower = df_auto.horsepower.astype(int)
df_auto.bore = df_auto.bore.astype(float)
df_auto.stroke = df_auto.stroke.astype(float)
df_auto.peak_rpm = df_auto.peak_rpm.astype(int)
df_auto.price = df_auto.price.astype(int)
df_auto = df_auto.select_dtypes(include=np.number)

In [ ]:
df_auto.info()
df_auto_desc = df_auto.select_dtypes(include=np.number).describe()
display(df_auto_desc)
df_auto_cr = df_auto.select_dtypes(include=np.number).corr()
display(df_auto_cr)

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(df_auto['curb_weight'], df_auto['price'], color='darkblue');

# 2. Data Regression

## 2.1 General Analysis variable/target Separation

In [ ]:
# Separation des variables explicatives (features) et de la variable à prédire (target)
data = df_auto.drop('price', axis=1)
target = df_auto['price']

In [ ]:
# Séparation de données d'entrainement et données de test
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=789)
print("Train Set:", X_train.shape)
print("Test Set:", X_test.shape)

## 2.2 Linear Regression (univariée)

In [ ]:
# Definition et Entrainement du modèle
regLR = LinearRegression()
regLR.fit(X_train[['curb_weight']], y_train)

In [ ]:
# Evaluation du modèle sur les données d'entrainement
print("Score R² calculé par le modèle:", regLR.score(X_train[['curb_weight']], y_train))

In [ ]:
# Récupération des données d'ajustement pour une régression simple
print("l'ordonnée à l'origine (intercept) calculée par le modèle:", regLR.intercept_)
print("la pente (coeff) de la droite (modèle univarié):",regLR.coef_)

In [ ]:
# Validation croisée entre 4 sous échantillons d'entrainement pour la régression (score MSE / RMSE / MAE / R²)
scores = cross_validate( # entraine et évalue le modèle sur chaque groupe (parametre cv)
    regLR, X_train[['curb_weight']], y_train, 
    cv=4, return_train_score=True,
    scoring=['r2', 'neg_mean_squared_error','neg_root_mean_squared_error', 'neg_mean_absolute_error'])
print(
    f"Model: {regLR}\n"
    f"test Score (R²): {scores['test_r2'].mean().round(4)} "
    f"(+/- {scores['test_r2'].std().round(4)})\n"
    f"train Score (R²): {scores['train_r2'].mean().round(4)} "
    f"(+/- {scores['train_r2'].std().round(4)})\n"
    f"MSE Score: {scores['test_neg_mean_squared_error'].mean().round(0)} "
    f"(+/- {scores['test_neg_mean_squared_error'].std().round(0)})\n"
    f"RMSE Score: {scores['test_neg_root_mean_squared_error'].mean().round(0)} "
    f"(+/- {scores['test_neg_root_mean_squared_error'].std().round(0)})\n"
    f"MAE Score: {scores['test_neg_mean_absolute_error'].mean().round(0)} "
    f"(+/- {scores['test_neg_mean_absolute_error'].std().round(0)})"
)

In [ ]:
# Validation croisée avec prédiction via 4 sous échantillons sur les données d'entrainement
y_train_preds = cross_val_predict( # entraine et renvoie les prédictions sur chaque groupe (parametre cv) considéré comme données de test
    regLR, X_train[['curb_weight']], y_train,
    cv=4)
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets)**2).mean())
print(f"RMSE Score: {rmse(y_train_preds, y_train).round(0)}")

In [ ]:
# Prédiction du modèle sur les données
y_train_pred = regLR.predict(X_train[['curb_weight']])
y_test_pred = regLR.predict(X_test[['curb_weight']])

plt.figure(figsize=(8, 6))
plt.suptitle("Régression prédite appliquée sur les données")
plt.scatter(X_train[['curb_weight']], y_train, color='darkgreen')
plt.plot(X_train[['curb_weight']], y_train_pred, color='blue')
plt.scatter(X_test[['curb_weight']], y_test, color='darkblue')
plt.plot(X_test[['curb_weight']], y_test_pred, color='green');

In [ ]:
# Analyse et affichage des résidus sur les données d'entrainement et de test
y_test_residus = y_test_pred - y_test
y_test_residus_norm = (y_test_residus-y_test_residus.mean())/y_test_residus.std()

plt.figure(figsize=(8, 2))
plt.suptitle("Valeur des résidus en fonction de X_test")
plt.scatter(X_test[['curb_weight']], y_test_residus, color='red')
plt.plot([X_test.curb_weight.min(),X_test.curb_weight.max()], [0,0], color='black');

plt.figure(figsize=(8, 4))
plt.suptitle("Centrage et réduction des résidus et comparaison avec la bissectrice normale")
stats.probplot(y_test_residus_norm, plot=plt);

In [ ]:
# Evaluation du modèle sur les données de test
print("Score R² calculé par le modèle:", regLR.score(X_test[['curb_weight']], y_test))

In [ ]:
# Test statistique univarié sur chaque variable explicative de la cible (et sur les données totales)
# NB : cela ne prouve pas la causalité ni l'importance, juste la corrélation
f_statistics, p_values = f_regression(data, target)
for column, f, p in zip(data.columns, f_statistics, p_values):
    print (f"[{column}]\n [F-Stat : {f.round(2)}] [P-Value : {p.round(6)}]")

## 2.3 Linear Regression (multivariée)

### 2.3.1 Initiale

In [ ]:
# Definition et Entrainement du modèle
regLR_multi = LinearRegression()
regLR_multi.fit(X_train, y_train)

In [ ]:
# Récupération des données d'ajustement pour une régression simple
print("l'intercept calculé par le modèle:", regLR_multi.intercept_)
df_coeff = pd.DataFrame([(i, float(j.round(2))) for i, j in zip(X_test.columns,regLR_multi.coef_)])
print("les coeff du modèle multivarié:",df_coeff)

In [ ]:
# Evaluation du modèle sur les données d'entrainement
print("Score R² calculé par le modèle:", regLR_multi.score(X_train, y_train))

In [ ]:
# Prédiction du modèle sur les données de test
y_train_pred = regLR_multi.predict(X_train)
y_test_pred = regLR_multi.predict(X_test)

plt.scatter(y_train, y_train_pred, color='darkblue')
plt.plot([y_train.min(),y_train.max()], [y_train.min(),y_train.max()], 'b--')
plt.scatter(y_test, y_test_pred, color='darkgreen')
plt.plot([y_test.min(),y_test.max()], [y_test.min(),y_test.max()], 'g--');

In [ ]:
# Validation croisée entre 4 sous échantillons d'entrainement pour la régression (score MSE / RMSE / MAE / R²)
scores = cross_validate( # entraine et évalue le modèle sur chaque groupe (parametre cv)
    regLR_multi, X_train, y_train, 
    cv=4, return_train_score=True,
    scoring=['r2', 'neg_mean_squared_error','neg_root_mean_squared_error', 'neg_mean_absolute_error'])
print(
    f"Model: {regLR_multi}\n"
    f"test Score (R²): {scores['test_r2'].mean().round(4)} "
    f"(+/- {scores['test_r2'].std().round(4)})\n"
    f"train Score (R²): {scores['train_r2'].mean().round(4)} "
    f"(+/- {scores['train_r2'].std().round(4)})\n"
    f"MSE Score: {scores['test_neg_mean_squared_error'].mean().round(0)} "
    f"(+/- {scores['test_neg_mean_squared_error'].std().round(0)})\n"
    f"RMSE Score: {scores['test_neg_root_mean_squared_error'].mean().round(0)} "
    f"(+/- {scores['test_neg_root_mean_squared_error'].std().round(0)})\n"
    f"MAE Score: {scores['test_neg_mean_absolute_error'].mean().round(0)} "
    f"(+/- {scores['test_neg_mean_absolute_error'].std().round(0)})"
)

In [ ]:
# Evaluation du modèle sur les données de test
print("Score R² calculé par le modèle:", regLR_multi.score(X_test, y_test))

In [ ]:
# Analyse et affichage des résidus sur les données
y_test_residus = y_test_pred - y_test
y_test_residus_norm = (y_test_residus-y_test_residus.mean())/y_test_residus.std()
y_train_residus = y_train_pred - y_train
y_train_residus_norm = (y_train_residus-y_train_residus.mean())/y_train_residus.std()

plt.figure(figsize=(8, 2))
plt.suptitle("Valeur des résidus en fonction de y_test")
plt.scatter(y_train, y_train_residus, color='darkblue')
plt.scatter(y_test, y_test_residus, color='darkgreen')
plt.plot([y_train.min(),y_train.max()], [0,0], color='blue')
plt.plot([y_test.min(),y_test.max()], [0,0], color='green');

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
stats.probplot(y_train_residus_norm, plot=axes[0])
axes[0].set_title("QQ-Plot : residus y_train")
axes[0].get_lines()[0].set_color('darkblue') 
axes[0].get_lines()[1].set_color('blue') 
stats.probplot(y_test_residus_norm, plot=axes[1])
axes[1].set_title("QQ-Plot : residus y_test")
axes[1].get_lines()[0].set_color('darkgreen') 
axes[1].get_lines()[1].set_color('green') 
plt.tight_layout()
plt.show()

### 2.3.1 Affinage manuel

In [ ]:
# Visualisation de la correlation entre les variables explicatives (avec seaborn)
plt.figure(figsize=(13, 13))
sns.heatmap(df_auto.select_dtypes(include='number').corr(), annot=True, cmap="RdBu_r", center=0)
plt.tight_layout();
sns.pairplot(df_auto[['curb_weight', 'horsepower', 'highway_mpg', 'height', 'bore', 'width', 'price']]);

In [ ]:
# Affinage du modèle manuel avec un sous ensemble de features après analyse des graphiques de correlation
signif_features = ['curb_weight', 'horsepower', 'bore', 'width']
regLR_multi_man = LinearRegression()
regLR_multi_man.fit(X_train[signif_features], y_train)
print("Score R² train:", regLR_multi_man.score(X_train[signif_features], y_train))
print("Score R² test:", regLR_multi_man.score(X_test[signif_features], y_test))

### 2.3.2 Affinage par test statistique

In [ ]:
# Affinage du modèle avec un sous ensemble déterminé par score f_regression (test statistique)
skb = SelectKBest(score_func=f_regression, k=3)
skb.fit(X_train, y_train)
print("Features Significatives:", X_train.columns[skb.get_support()])
regLR_multi_skb = LinearRegression()
regLR_multi_skb.fit(skb.transform(X_train), y_train)
print("Score R² train:", regLR_multi_skb.score(skb.transform(X_train), y_train))
print("Score R² test:", regLR_multi_skb.score(skb.transform(X_test), y_test))

### 2.3.3 Affinage par sélection via le poids des variables

In [ ]:
# Affinage du modèle avec une selection depuis le poids (coefficients) des variables dans le modèle
regLR_multi_init = LinearRegression()
sfm = SelectFromModel(regLR_multi_init)
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_sfm = sfm.fit_transform(X_train_scaled, y_train)
X_test_sfm = sfm.transform(X_test_scaled)
print("Features Significatives:", X_train.columns[sfm.get_support()])
regLR_multi_sfm = LinearRegression()
regLR_multi_sfm.fit(X_train_sfm, y_train)
print("Score R² train:", regLR_multi_sfm.score(X_train_sfm, y_train))
print("Score R² test:", regLR_multi_sfm.score(X_test_sfm, y_test))

### 2.3.4 Affinage par Ridge
- Conserve les coefficients mais réduit très fortement ceux des variables peu corrélées (somme des carrée)
- Dense (toutes les variables) mais peu sensible aux correlations entre variables
- nécessite un centrage réduction (scaler) pour limiter les effets des variables dont les valeurs ont des ranges important (somme des carré oblige)

In [ ]:
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
regLR_multi_RCV = RidgeCV(alphas=(0.001, 0.01, 0.1, 0.3, 0.7, 1.0, 10.0, 50.0, 100.0))
regLR_multi_RCV.fit(X_train_scaled, y_train)
print("Alpha retenu par cross validation:", regLR_multi_RCV.alpha_)
print("Score R² train:", regLR_multi_RCV.score(X_train_scaled, y_train))
print("Score R² test:", regLR_multi_RCV.score(X_test_scaled, y_test))
y_train_pred = regLR_multi_RCV.predict(X_train_scaled)
y_test_pred  = regLR_multi_RCV.predict(X_test_scaled)
print("Score MSE train:", mean_squared_error(y_train, y_train_pred))
print("Score MSE test:", mean_squared_error(y_test, y_test_pred))

### 2.3.5 Affinage par Lasso
- Peut annuler des coefficients (somme des valeurs absolues)
- Peu dense (peu de variables) mais sensible aux correlations entre variables

In [ ]:
regLR_multi_L = Lasso(alpha=1)
regLR_multi_L.fit(X_train, y_train)
df_coeff = pd.DataFrame([(i, float(j.round(2))) for i, j in zip(X_test.columns,regLR_multi_L.coef_)])
print(df_coeff)
plt.plot(df_coeff[0], df_coeff[1])
plt.suptitle('Valeur des coefficient par variable')
plt.xticks(rotation=70);
print("Score R² train:", regLR_multi_L.score(X_train, y_train))
print("Score R² test:", regLR_multi_L.score(X_test, y_test))
y_train_pred = regLR_multi_L.predict(X_train)
y_test_pred  = regLR_multi_L.predict(X_test)
print("Score MSE train:", mean_squared_error(y_train, y_train_pred))
print("Score MSE test:", mean_squared_error(y_test, y_test_pred))

In [ ]:
regLR_multi_LCV = LassoCV(cv=10)
regLR_multi_LCV.fit(X_train, y_train)
alpha = regLR_multi_LCV.alpha_
alphas = regLR_multi_LCV.alphas_
mse_matrix = regLR_multi_LCV.mse_path_
mse_mean = mse_matrix.mean(axis=1)
plt.figure(figsize = (10, 8))
for i in range(mse_matrix.shape[1]):  # boucle sur les folds (cv)
    plt.plot(alphas, mse_matrix[:, i], ':', label=f'MSE échantillon {i}')
plt.plot(alphas, mse_mean, 'r-', label = 'MSE Moyen')
plt.axvline(x=alpha, color='black', lw=1, ls='--', label = f'Alpha retenu {alpha.round(0)}')
plt.xlabel('Alpha')
plt.ylabel('Mean square error')
plt.title('Mean square error pour chaque échantillon')
plt.legend()
plt.tight_layout();
print("Score R² train:", regLR_multi_LCV.score(X_train, y_train))
print("Score R² test:", regLR_multi_LCV.score(X_test, y_test))
y_train_pred = regLR_multi_LCV.predict(X_train)
y_test_pred  = regLR_multi_LCV.predict(X_test)
print("Score MSE train:", mean_squared_error(y_train, y_train_pred))
print("Score MSE test:", mean_squared_error(y_test, y_test_pred))